### Строим логистическую регрессию - угадываем пол спортсмена по признакам

https://www.kaggle.com/rio2016/olympic-games

In [1]:
import pandas as pd
import numpy as np
#целевая переменная - столбец пол.

In [2]:
data = pd.read_csv( 'athletes.csv' )
data.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11538 non-null  int64  
 1   name         11538 non-null  object 
 2   nationality  11538 non-null  object 
 3   sex          11538 non-null  object 
 4   dob          11537 non-null  object 
 5   height       11208 non-null  float64
 6   weight       10879 non-null  float64
 7   sport        11538 non-null  object 
 8   gold         11538 non-null  int64  
 9   silver       11538 non-null  int64  
 10  bronze       11538 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 991.7+ KB


#### Попробуем угадать пол на основе роста, веса и вида спорта

Посмотрим много ли в наших признаках пустых значений

In [4]:
data[ pd.isnull( data['height'] ) ].head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
12,258556239,Abbas Qali,IOA,male,10/11/92,NaN,NaN,aquatics,0,0,0
47,469953606,Abdoullah Bamoussa,ITA,male,6/8/86,NaN,NaN,athletics,0,0,0
50,325809293,Abdul Omar,GHA,male,10/3/93,NaN,NaN,boxing,0,0,0
52,262868423,Abdulaziz Alshatti,IOA,male,10/30/90,NaN,NaN,fencing,0,0,0
56,897549624,Abdullah Hel Baki,BAN,male,8/1/89,NaN,NaN,shooting,0,0,0


In [5]:
print('Для height пустых строк {}'.format( len( data[ pd.isnull( data['height'] ) ] ) ))
print('Для weight пустых строк {}'.format( len( data[ pd.isnull( data['weight'] ) ] ) ))
print('Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sport'] ) ] ) ))
print('Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sex'] ) ] ) ))
print('Всего строк в наборе {}'.format( len( data ) ))

Для height пустых строк 330
Для weight пустых строк 659
Для sport пустых строк 0
Для sport пустых строк 0
Всего строк в наборе 11538


In [6]:
type(data['sex'][0])

str

In [7]:
temp = pd.DataFrame(0.0, columns = pd.MultiIndex.from_product([data['sex'].unique(), ['weight', 'height']]), index = data['sport'].unique() )

In [8]:
for sport in temp.index:
    for sex in ('male', 'female'):
        for x in ('weight', 'height'):
            temp.loc[sport][sex, x] = data[data['sport'] == sport][data['sex'] == sex][[x]].mean(skipna=True)

C:\Users\BIASPA~1\AppData\Local\Temp/ipykernel_19116/1097964379.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp.loc[sport][sex, x] = data[data['sport'] == sport][data['sex'] == sex][[x]].mean(skipna=True)


In [9]:
temp.fillna(0, inplace=True)

In [10]:
temp.head()

male               female          
              weight    height     weight    height
athletics  74.777680  1.809234  60.152542  1.690500
fencing    78.785124  1.833802  62.733871  1.712581
taekwondo  74.809524  1.863750  61.258065  1.730159
cycling    72.576052  1.793074  60.207254  1.673128
triathlon  66.814815  1.788704  54.563636  1.676545

In [11]:
for i in data.index:
    t = data.loc[i]
    if (pd.isnull(t['height'])):
        data.loc[i, 'height'] = temp.loc[t['sport']][t['sex'], 'height']
    if (pd.isnull(t['weight'])):
        data.loc[i, 'weight'] = temp.loc[t['sport']][t['sex'], 'weight']        

In [12]:
print('Для sex пустых строк {}'.format( len( data[ pd.isnull( data['sex'] ) ] ) ))
print('Для height пустых строк {}'.format( len( data[ pd.isnull( data['height'] ) ] ) ))
print('Для weight пустых строк {}'.format( len( data[ pd.isnull( data['weight'] ) ] ) ))
print('Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sport'] ) ] ) ))
print('Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sex'] ) ] ) ))
print('Всего строк в наборе {}'.format( len( data ) ))

Для sex пустых строк 0
Для height пустых строк 0
Для weight пустых строк 0
Для sport пустых строк 0
Для sport пустых строк 0
Всего строк в наборе 11538


In [13]:
data.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11538 non-null  int64  
 1   name         11538 non-null  object 
 2   nationality  11538 non-null  object 
 3   sex          11538 non-null  object 
 4   dob          11537 non-null  object 
 5   height       11538 non-null  float64
 6   weight       11538 non-null  float64
 7   sport        11538 non-null  object 
 8   gold         11538 non-null  int64  
 9   silver       11538 non-null  int64  
 10  bronze       11538 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 991.7+ KB


In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[ [ 'height', 'weight', 'sport', 'sex' ] ]

# столбец sport является категориальной переменой
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'sport' ] )

# столбец sex является целевой переменной, удаляем его из X
del X['sex']
X.head()

,height,weight,sport_aquatics,sport_archery,sport_athletics,sport_badminton,sport_basketball,sport_boxing,sport_canoe,sport_cycling,...,sport_rugby sevens,sport_sailing,sport_shooting,sport_table tennis,sport_taekwondo,sport_tennis,sport_triathlon,sport_volleyball,sport_weightlifting,sport_wrestling
0,1.72,64.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.68,56.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.98,79.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.83,80.0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.81,71.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# целевая переменная (столбец sex) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [18]:
le.fit( data['sex'] )

LabelEncoder()

In [19]:
le.classes_

array(['female', 'male'], dtype=object)

In [20]:
# пример "расшировки" столбца sex

le.transform( [ 'male', 'female', 'male' ] )

array([1, 0, 1])

In [21]:
# записываем в переменную y преобразованный столбец sex

y = pd.Series( data = le.transform( data['sex'] ) )
y.head()

0    1
1    0
2    1
3    1
4    1
dtype: int32

In [22]:
model = LogisticRegression(max_iter=1000)

In [23]:
# обучаем модель
model.fit( X, y )
predictions = model.predict_proba( X )

In [24]:
predictions[:5]

array([[0.5857263 , 0.4142737 ],
       [0.90604676, 0.09395324],
       [0.02059745, 0.97940255],
       [0.17037628, 0.82962372],
       [0.1485224 , 0.8514776 ]])

In [28]:
# сравниваем факт с предсказаниями
# (да, сравнивать предсказания с самими данными не круто, лучше было разбить на обучающую и тестовую выборку)
# это будет следующим заданием

list(zip( predictions[:, 1], y ))[:5]

[(0.41427369992115404, 1),
 (0.09395323766593203, 0),
 (0.9794025509670963, 1),
 (0.829623716487017, 1),
 (0.8514775954641487, 1)]

In [29]:
model.score(X, y)

0.8219795458485006

### Задание

1. Построить модель. 
2. Улучшить ее показатели

In [27]:
len(model.coef_[0])
len(X.columns)

30